In [1]:
import pandas as pd
import numpy as np

In [2]:
import pyodbc

# Connection details
server = '10.147.17.61'     
database = 'MESinterface'     
username = 'JKBTP_USER'    
password = 'JK@$userbtp'  

# Build connection string
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
)

# Create connection
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

print("Connected successfully!")

Connected successfully!


In [3]:
query = """
SELECT
    Lot_Id,
    UOM,
    Qty,
    MaterialCode,
    LiveQty,
    dtandtime,
    'TBMStage1' AS source_schema
FROM TBMStage1.I_Material

UNION ALL

SELECT
    Lot_Id,
    UOM,
    Qty,
    MaterialCode,
    LiveQty,
    dtandtime,
    'TBMStage2' AS source_schema
FROM TBMStage2.I_Material;
"""

df_material = pd.read_sql(query, conn)


/var/folders/v_/jlq85y5x7hj9pzv0lg1c9lfh0000gn/T/ipykernel_55146/2491553772.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_material = pd.read_sql(query, conn)


In [4]:
from datetime import datetime
import pandas as pd

# ===============================
# CUSTOM DATE-TIME INPUTS
# ===============================
start_datetime = "2025-12-31 22:00:00"
end_datetime   = "2025-12-31 23:59:59"

print("Start DateTime:", start_datetime)
print("End DateTime  :", end_datetime)


query = f"""
SELECT
    Lot_Id,
    UOM,
    Qty,
    MaterialCode,
    LiveQty,
    dtandtime,
    'TBMStage1' AS source_schema
FROM TBMStage1.I_Material
WHERE dtandtime >= '{start_datetime}'
  AND dtandtime <  '{end_datetime}'

UNION ALL

SELECT
    Lot_Id,
    UOM,
    Qty,
    MaterialCode,
    LiveQty,
    dtandtime,
    'TBMStage2' AS source_schema
FROM TBMStage2.I_Material
WHERE dtandtime >= '{start_datetime}'
  AND dtandtime <=  '{end_datetime}';
"""

try:
    df_material = pd.read_sql(query, conn)
    print(f"✅ Retrieved {len(df_material)} rows from I_Material (Stage1 + Stage2)")
except Exception as e:
    print(f"❌ Error while fetching I_Material data: {e}")


Start DateTime: 2025-12-31 22:00:00
End DateTime  : 2025-12-31 23:59:59


/var/folders/v_/jlq85y5x7hj9pzv0lg1c9lfh0000gn/T/ipykernel_55146/855917229.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_material = pd.read_sql(query, conn)


✅ Retrieved 800 rows from I_Material (Stage1 + Stage2)


,Lot_Id,UOM,Qty,MaterialCode,LiveQty,dtandtime,source_schema
31,CapStripNew_31122025231117_1157,None,2070,CAPSTRIP - 10MM WIDTH,0.00,2025-12-31 23:37:17.330,TBMStage2
32,1049085353,None,1,CAR155R13ULTXPC1TL,0.00,2025-12-31 22:08:55.310,TBMStage2
33,1049085361,None,1,CAR155R13ULTXPC1TL,0.00,2025-12-31 22:26:00.453,TBMStage2
34,1049085369,None,1,CAR155R13ULTXPC1TL,0.00,2025-12-31 22:35:03.980,TBMStage2
35,1049085377,None,1,CAR155R13ULTXPC1TL,0.00,2025-12-31 22:42:54.917,TBMStage2
...,...,...,...,...,...,...,...
795,1048396689,None,1,CAR1458012HILIFETT,0.00,2025-12-31 22:10:06.403,TBMStage2
796,1048396681,None,1,CAR1458012HILIFETT,0.00,2025-12-31 22:17:55.230,TBMStage2
797,WBCNew_31122025132446_6,None,347.156,CPJ1218-114MM/29°,101.00,2025-12-31 23:27:38.713,TBMStage2
798,CapStripNew_31122025093836_965,None,2173.5,CAPSTRIP - 10MM WIDTH,0.00,2025-12-31 22:48:38.343,TBMStage2


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

# -----------------------
# 1. MySQL Connection
# -----------------------
host = "35.208.174.2"
user = "root"
password = "Dev112233"
database = "jkplanning_CTB"

engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}/{database}",
    pool_pre_ping=True
)

table_name = "I_Material_Combined"

# -------------------------------------------------
# 2. SANITY CHECK (RECOMMENDED)
# -------------------------------------------------
print("✅ Rows to upload:", len(df_material))
print("✅ Columns:", df_material.columns.tolist())
print(df_material.head(3))

# -------------------------------------------------
# 3. TRUNCATE + UPLOAD
# -------------------------------------------------
with engine.begin() as conn:
    #  Remove all old data
    conn.execute(text(f"TRUNCATE TABLE {table_name};"))
    print("🗑️ Old I_Material_Combined data removed")

    #  Upload fresh data
    df_material.to_sql(
        table_name,
        conn,
        if_exists="append",
        index=False,
        chunksize=1000,     # safe for large tables
        method="multi"      # fast insert
    )

print("✅ Fresh I_Material_Combined data uploaded successfully.")
